# Light Traffic Detection
---
[image1]: ./data/results/ssd_frcnn.jpg
[image2]: ./data/results/real_ssd.jpg

## Overview

This project is an implementation for Light Traffic Detection as a part of SDCND-Capstone project of Blindspot-team. The implementation is done by using TensorFlow API Object Detection to use pre-trained model and re-train the model on Similutaor and real track data.

## TensorFlow Installation

I would recommend to read the following resources for API's installation steps to get deep insight about the steps and how to tackle the challenges [TesorFlow Doc](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md), [Alex Lencher](https://github.com/alex-lechner/Traffic-Light-Classification), [Daniel Stang](https://medium.com/@WuStangDan/step-by-step-tensorflow-object-detection-api-tutorial-part-1-selecting-a-model-a02b6aabe39e). 

1. Install Tensorflow 1.4 to make the project compatible with the Carla's TensorFlow version.
```
pip install tensorflow==1.4
pip install tensorflow-gpu==1.4
```
2. Install the following dependencies.
```
sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
sudo pip install Cython
sudo pip install jupyter
sudo pip install matplotlib
```
3. Clone TensorFlow's models repository in where in your computer.
```
git clone https://github.com/tensorflow/models.git
```
4. the most important thing to make the project compatible with carla's TensorFlow Version (1.3) after cloning the TensorFlow models you must navigate to `models` directory and excute the following command.
```
git checkout f7e99c0
```
5. Navigate to `models/research` and Execute the following command.
```
protoc object_detection/protos/*.proto --python_out=.
```
6. Add Libraries to PYTHONPATH by the following command. This command needs to run from every new terminal you start. If you wish to avoid running this manually, you can add it as a new line to the end of your ~/.bashrc file.
```
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
```
7. Test the installation and if it didn't showed any error so the API installed correctly.
```
python object_detection/builders/model_builder_test.py
```

## Data Prepration
I was provided the data by the team leader Dario with the labels `.yaml` file for the simulator and real data. I used [ Anthony Sarkis'](https://github.com/swirlingsand/deeper-traffic-lights/blob/master/data_conversion_udacity_real_large.py) script to create TFRecord file for the training data. I changed the labels dictrionary to 4 class for the simultor data and real data rather than 14.

```
LABEL_DICT =  {
    "Green" : 1,
    "Red" : 2,
    "Yellow" : 3,
    "off" : 4,

    }
    
```


and I changed the path of the `.yaml` fil and ther run the following command.
```
python tf_record_sim.py --output_path training_sim.record
```

## Traning
For training ``I used two models SSD_inception_V2 and faster_rcnn_inception_v2_coco from[Tensorflow detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md). 
```After downloading the model and modification the configration in the .config of each models as following.
```
 num_classes: 4
```
```
  fine_tune_checkpoint: "models/model.ckpt"
  from_detection_checkpoint: true
  num_steps= 20000
```
train_input_reader: {
  tf_record_input_reader {
    input_path: "data/dataset-sdcnd-capstone-train/real_data_train.record"
  }
  label_map_path: "data/label_map-sim.pbtxt"
}

eval_config: {
  num_examples: 485
  max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "data/dataset-sdcnd-capstone-test/real_data_test.record"
  }
  label_map_path: "data/label_map-sim.pbtxt"
  shuffle: false
  num_readers: 1
  num_epochs: 1
}
```
`train.py` is the file that has the script to retrain the models on the project's data with this command.
```
python train.py --logtostderr --train_dir=./models/train/sim_ssd --pipeline_config_path=ssd_inception_v2_coco_sim.config

```



```
python export_inference_graph.py --input_type image_tensor --pipeline_config_path ./ssd_inception_v2_coco_sim.config --trained_checkpoint_prefix ./models/train/sim_ssd/model.ckpt-30000 --output_directory ./fine_tuned_model/sim_ssd

```
## Result
The SSD_Inception was chosen for the project for it's perfomance and processing time. Fast-RCNN was better to detect small boxes however it was verys slower to process Image in 3 Sec and SSD_Inception has process time around .2 sec. and the both proced the following result.

![title][image1]
![title][image2]